# 라이브러리 설치

In [18]:
# 설치할거 전부
!pip install -q chromadb sentence-transformers openai ipywidgets
# !pip install -q langchain_core
# import sys
# !{sys.executable} -m pip install -q ipywidgets
# !pip install -q langchain langchain-community langchain-core langchain_openai
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


# API Key 환경 설정

In [2]:
# Hugging Face 및 Google API 인증
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN') # Hugging Face 액세스 토큰
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY') # Google Custom Search API 키
GOOGLE_CSE_ID = userdata.get('GOOGLE_CSE_ID') # Programmable Search Engine ID
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY') # OpenAI API Key

# Base Knowledge 수집용 네이버쇼핑 영양제 상품 검색 코드

In [3]:
# 네이버쇼핑 검색 로직
import requests
from urllib.parse import quote

client_id = "SWe77FM8c_EjAm5Y93h5"
client_secret = "mAz_OvLf7T"

def naver_shop_search(query, display=100):
    encoded_query = quote(query)
    url = f"https://openapi.naver.com/v1/search/shop.json?query={encoded_query}&display={display}&start=1&sort=sim&exclude=used:cbshop:rental"

    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        return {"error": response.status_code}

result = naver_shop_search("영양제", 100)

# 추가 정보 수집을 위한 구글 검색 코드

In [4]:
# Google Programmable Search Engine 설정
from googleapiclient.discovery import build

def google_search(query, num=10):
    service = build("customsearch", "v1", developerKey=GOOGLE_API_KEY)
    res = service.cse().list(q=query, cx=GOOGLE_CSE_ID, num=num).execute()
    return [item["snippet"] for item in res["items"]]


# Base Knowledge를 Vector DB에 저장하는 코드

In [5]:
# 네이버쇼핑 검색 결과 -> Vector DB (Chroma) 저장
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer

# 데이터 전처리
def preprocess_items(items):
    processed = []
    for item in items:
        data = {
            "id": item['productId'],
            "text": f"{item['title']} {item['brand']} {item['maker']} {item['category3']}",  # 임베딩용 텍스트
            "metadata": {
                "price": int(item['lprice']),
                "brand": item['brand'],
                "category": f"{item['category1']}>{item['category2']}>{item['category3']}",
                "link": item['link']
            }
        }
        processed.append(data)
    return processed

# Chroma DB 초기화
client = chromadb.PersistentClient(path="./naver_shopping_db")
embedding_model = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
)
# 기존 컬렉션이 있으면 삭제 후 재생성
try:
    client.delete_collection("products")
except:
    pass
collection = client.create_collection(
    name="products",
    embedding_function=embedding_model,
    metadata={"hnsw:space": "cosine"}
)

# 데이터 저장
if 'items' in result:
    processed_data = preprocess_items(result['items'])
    collection.add(
        ids=[item['id'] for item in processed_data],
        documents=[item['text'] for item in processed_data],
        metadatas=[item['metadata'] for item in processed_data]
    )
    print("성공적으로", len(processed_data), "개 상품 저장됨")
else:
    print("Error:", result.get('error', '데이터 없음'))


성공적으로 100 개 상품 저장됨


In [18]:
# from langchain.agents import AgentExecutor, create_openai_tools_agent
# from langchain_openai import ChatOpenAI
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# from langchain_core.tools import tool

# # 2. 검색 도구 정의 (Function Calling)
# tool
# def search_products(query: str, max_results: int = 5) -> list:
#     """제품 검색을 위해 Chroma DB를 쿼리하는 도구"""
#     """
#     Chroma DB를 쿼리하여 제품 정보를 검색하는 도구

#     Parameters:
#     - query (str): 제품 검색 질의어입니다. 예: "비타민C 고함량"
#     - max_results (int): 반환할 최대 제품 개수입니다. 기본값은 3개입니다.

#     Returns:
#     - list: 각 제품의 정보(제품명, 가격, 링크)를 담은 딕셔너리들의 리스트
#       예시: [{"제품": "제품명", "가격": 10000, "링크": "https://..."}, ...]
#     """
#     results = collection.query(
#         query_texts=[query],
#         n_results=max_results
#     )
#     return [{
#         "제품": doc,
#         "가격": meta['price'],
#         "링크": meta['link']
#     } for doc, meta in zip(results['documents'][0], results['metadatas'][0])]

# tool
# def google_search_tool(query: str, num_results: int = 10) -> list:
#     """구글 검색을 통해 제품 상세 정보 및 최신 논문을 찾는 도구. 질문 답변에 필요한 웹 기반 정보 수집에 사용"""
#     """
#     구글 검색 API를 통해 웹에서 정보를 검색하는 도구

#     Parameters:
#     - query (str): 검색할 질의어(검색어)입니다. 예: "2025년 영양제 트렌드"
#     - num_results (int): 반환할 최대 검색 결과 개수입니다. 기본값은 3개입니다.

#     Returns:
#     - list: 각 검색 결과의 요약(snippet) 텍스트 리스트
#     """
#     return google_search(query, num=num_results)

# # 3. 에이전트 설정
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3, openai_api_key=OPENAI_API_KEY)
# llm_with_tools = llm.bind_tools([search_products, google_search_tool])

# system_prompt = """당신은 영양제 전문 약사 어시스턴트입니다. 다음 규칙을 준수하세요:
# - 사용자 질문 유형에 따라 적절한 도구 선택
#   - 제품 검색: search_products
#   - 제품 상세 정보, 최신 논문: google_search_tool
# - 검색 결과를 바탕으로 정확한 정보 제공
# - 추천하는 이유를 자세하게 설명
# - 가격, 성분 비교시 표 형태로 응답
# - 모르는 정보는 확대 해석하지 말 것
# """

# prompt = ChatPromptTemplate.from_messages([
#     ("system", system_prompt),
#     ("human", "{input}"),
#     MessagesPlaceholder("agent_scratchpad")
# ])

# tools = [search_products]
# agent = create_openai_tools_agent(llm_with_tools, tools, prompt)
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False) # verbose=True 로 변경안해도 됨


In [ ]:
# # 4. 실행 예시
# response = agent_executor.invoke({
#     "input": "종종 머리가 아프고 눈이 피곤해. 어떤 영양제를 먹으면 좋을지 추천해줘. 주요 성분과 함량도 알려줘."
# })
# print(response['output'])

In [22]:
# !pip install -q ipywidgets

# !pip install -q openai ipywidgets requests
# import sys
# # !{sys.executable} -m pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


# 검색 도구 함수 정의 (Function Calling)

In [29]:
# from langchain.tools import tool
# from langchain_core.utils.function_calling import convert_to_openai_function

recommend_nutrient = {
    "name": "recommend_nutrient",
    "description": "반드시 첫 번째로 호출해야 하는 필수 함수입니다. 사용자 증상을 분석해 영양소를 추천하거나 제거할 것을 결정합니다.반드시 nutrient와 action 필드를 포함해야 합니다.",
    "parameters": {
        "type": "object",
        "properties": {
            "nutrient": {"type": "string", "description": "추천하거나 제거가 필요한 영양소 성분"},
            "action": {"type": "string", "description": "'추천' 또는 '제거' 중 하나", "enum": ["추천", "제거"]},
            "query": {"type": "string", "description": "사용자 증상 쿼리"
            }
        },
        "required": ["nutrient", "action", "query"],
        "additionalProperties": False
    }
}

# tool
# def vectordb_search(query: str, max_results: int = 10) -> list:
#     """두 번째 단계에서 호출해야 합니다. 추천 영양소로 제품 검색"""
#     results = collection.query(
#         query_texts=[query],
#         n_results=max_results
#     )
#     return [{
#         "제품": doc,
#         "가격": meta['price'],
#         "링크": meta['link']
#     } for doc, meta in zip(results['documents'][0], results['metadatas'][0])]

vectordb_search = {
  "name": "vectordb_search",
  "description": "두 번째 단계에서 호출해야 합니다. 추천 영양소로 제품 검색",
  "parameters": {
    "type": "object",
    "properties": {
      "query": {"type": "string"},
      "max_results": {"type": "integer", "default": 10}
    },
    "required": ["query"]
  }
}
#tool
# def google_search(query: str, num_results: int = 10) -> list:
#     """세 번째 단계에서 호출해야 합니다. 검색된 제품의 상세 정보 조회"""
#     return google_search(query, num=num_results)

google_search = {
  "name": "google_search",
  "description": "세 번째 단계에서 호출해야 합니다. 검색된 제품의 상세 정보 조회",
  "parameters": {
    "type": "object",
    "properties": {
      "query": {"type": "string"},
      "num_results": {"type": "integer", "default": 10}
    },
    "required": ["query"]
  }
}
# tools를 OpenAI 함수 스펙으로 변환
# functions = [
#     recommend_nutrient,
#     convert_to_openai_function(vectordb_search),
#     convert_to_openai_function(google_search)
# ]


# LLM 동작 코드

In [28]:
from openai import OpenAI
import json
import re
from openai.types.chat import ChatCompletionMessageParam


client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

def get_nutrient_action(symptom, gender, age, pregnancy, mode):
    prompt = f"""
    사용자 정보:
    - 성별: {gender}
    - 나이: {age}
    - 임신 여부: {pregnancy}
    - 선택 기능: {mode}

    증상: {symptom}

위 조건을 고려하여 사용자에게 가장 적절한 영양소를 1가지 추천하거나, 과다 복용 시 제거가 필요한 영양소 1가지를 찾아.
결과는 Function Calling 형식으로 반환해줘.

    """
    res = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "너는 건강 기반의 영양제 조언 전문가야."},
            {"role": "user", "content": prompt}
        ],
        functions= [{"name": "recommend_nutrient"},{"name": "vectordb_search"},{"name": "google_search"}],
        # function_call={"name": "recommend_nutrient"},
        function_call="auto",
        temperature=0.5
    )
    args = res.choices[0].message.function_call.arguments
    parsed = json.loads(args)

    # LLM 응답 전체 출력
    print("=== 전체 응답 ===")
    print(res)

    # 함수 호출 내용 출력
    if res.choices[0].message.function_call:
        print("\n=== 함수 호출 정보 ===")
        print("함수 이름:", res.choices[0].message.function_call.name)
        print("함수 인자:", res.choices[0].message.function_call.arguments)

        # 인자 파싱
        try:
            parsed = json.loads(res.choices[0].message.function_call.arguments)
            print("\n=== 파싱된 인자 ===")
            print(json.dumps(parsed, indent=2, ensure_ascii=False))
        except json.JSONDecodeError:
            print("❗ 인자 파싱 실패")

    return parsed.get("nutrient", "비타민"), parsed.get("action", "추천")


def get_nutrient_action2(symptom, gender, age, pregnancy, mode):
    prompt = f"""
    사용자 정보:
    - 성별: {gender}
    - 나이: {age}
    - 임신 여부: {pregnancy}
    - 선택 기능: {mode}

    증상: {symptom}

[단계 1] 증상을 분석해 recommend_nutrient 함수를 반드시 호출하세요.
- 반드시 'nutrient'와 'action' 필드를 포함해야 합니다.
- 'action'은 '추천' 또는 '제거' 중 하나여야 합니다.
[단계 2] 추천 영양소를 포함한 제품을 vectordb_search로 5개 검색.
[단계 3] 검색된 제품을 google_search로 상세 정보 수집.
각 단계를 순차적으로 실행하고, 최종 결과를 표로 정리하세요.
    """
    messages = [
        {
            "role": "system",
            "content": "너는 건강 기반의 영양제 조언 전문가야. 반드시 첫 단계에서는 recommend_nutrient 함수를 호출해야 합니다."
        },
        {"role": "user", "content": prompt}
    ]

    max_steps = 3
    current_step = 0
    nutrient = None
    action = None

    while current_step < max_steps:
        print(f"\n=== STEP {current_step+1} ===")
        print("현재 messages:", messages)
        try:
            res = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages,
                functions= [{"name": "recommend_nutrient"},{"name": "vectordb_search"},{"name": "google_search"}],
                function_call="auto",
                temperature=0.7
            )
        except Exception as e:
            print("API 호출 오류:", e)
            break

        print("LLM 전체 응답:", res)

        if not res.choices[0].message.function_call:
            print("함수 호출 없음")
            break

        func_name = res.choices[0].message.function_call.name
        print("호출된 함수:", func_name)
        print("함수 인자(raw):", res.choices[0].message.function_call.arguments)
        try:
            args = json.loads(res.choices[0].message.function_call.arguments)
        except Exception as e:
            print("함수 인자 파싱 오류:", e)
            break

        if func_name == "recommend_nutrient":
            if "nutrient" not in args or "action" not in args:
                print("nutrient/action 필드 누락")
                # 오류 메시지를 구체적으로 작성
                error_msg = """
                  ERROR: nutrient와 action 필드가 모두 필요합니다.
                  예시: {"nutrient": "비타민C", "action": "추천"}
                  """
                messages.append({
                    "role": "user",
                    "content": error_msg
                })
                current_step += 1
                continue
            nutrient = args["nutrient"]
            action = args["action"]
            print(f"추천 영양소: {nutrient} ({action})")
            messages.append({
                "role": "assistant",
                "content": None,
                "function_call": {
                    "name": func_name,
                    "arguments": json.dumps(args)
                }
            })
            messages.append({
                "role": "user",
                "content": f"이제 {nutrient}을 포함한 제품을 검색해주세요."
            })
        elif func_name == "vectordb_search":
            if not nutrient:
                print("ERROR: 먼저 recommend_nutrient를 호출해야 합니다.")
                messages.append({
                    "role": "user",
                    "content": "ERROR: 먼저 recommend_nutrient를 호출해야 합니다."
                })
                current_step += 1
                continue
            query = args.get("query", nutrient)
            print(f"제품 검색 쿼리: {query}")
            results = vectordb_search(query)
            print(f"검색 결과: {results}")
            messages.append({
                "role": "assistant",
                "content": None,
                "function_call": {
                    "name": func_name,
                    "arguments": json.dumps(args)
                }
            })
            messages.append({
                "role": "user",
                "content": f"이제 {query} 제품의 상세 정보를 조회해주세요."
            })
        elif func_name == "google_search":
            query = args.get("query", "")
            print(f"구글 검색 쿼리: {query}")
            results = google_search(query)
            print(f"구글 검색 결과: {results}")
        else:
            print("알 수 없는 함수 호출:", func_name)
        current_step += 1

    print("최종 추천 영양소:", nutrient)
    print("최종 추천 액션:", action)
    return nutrient, action


# User Interface 정의 코드

In [20]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- 위젯 UI 구성 ---
gender = widgets.ToggleButtons(options=["남성", "여성"], description='성별:')
age = widgets.IntText(value=30, description='만 나이:')
pregnancy = widgets.ToggleButtons(options=["해당 없음", "임신 중", "임신 가능성"], description='임신 여부:')
mode = widgets.ToggleButtons(options=["식습관 분석을 통한 영양제 추천", "불편 현상에 따른 영양제 추천", "영양제 과다 복용 확인"], description='메뉴:')

# ✅ 입력창 너비 확장
symptom_input = widgets.Text(
    placeholder='증상 또는 복용 제품 입력',
    description='입력:',
    layout=widgets.Layout(width='95%')
)

submit_button = widgets.Button(description="제출", button_style="success")
output_box = widgets.Output()

# --- 실행 로직 ---
def on_submit_clicked(b):
    with output_box:
        clear_output()
        info = {
            "성별": gender.value,
            "나이": age.value,
            "임신 여부": pregnancy.value,
            "선택 기능": mode.value
        }
        symptom = symptom_input.value.strip()

        print(f"사용자 정보: {info}")
        print(f"선택한 기능: {mode.value}")
        print()

        if not symptom:
            print("⚠️ 증상이나 복용 제품을 입력해주세요.")
            return

        # nutrient, action = get_nutrient_action(symptom, gender.value, age.value, pregnancy.value, mode.value)
        # print(nutrient, action)
        get_nutrient_action2(symptom, gender.value, age.value, pregnancy.value, mode.value)

        # if action == "추천":
        #     print(f"🧾 '{symptom}'에 대한 AI 추천 결과:")
        #     print(f"- {nutrient} 성분이 포함된 제품이 적합할 수 있습니다.\n")
        #     products = naver_shop_search(nutrient)
        #     for p in products:
        #         title = re.sub(r"<.*?>", "", p['title'])
        #         print(f"• {title}")
        #         print(f"  - 브랜드: {p.get('brand', '정보 없음')}")
        #         print(f"  - 가격: {int(p['lprice']):,}원")
        #         print(f"  - 링크: {p['link']}\n")
        # else:
        #     print(f"⚠️ '{nutrient}' 성분이 중복 또는 과다 복용 위험이 있습니다.")
        #     print("🔍 해당 성분이 포함된 제품을 확인하고 복용 여부를 재검토하세요.\n")
        #     products = naver_shop_search(nutrient)
        #     for p in products:
        #         title = re.sub(r"<.*?>", "", p['title'])
        #         print(f"• {title}")
        #         print(f"  - 브랜드: {p.get('brand', '정보 없음')}")
        #         print(f"  - 가격: {int(p['lprice']):,}원")
        #         print(f"  - 링크: {p['link']}\n")

        print("※ 이 답변은 빅데이터 기반으로 제공되며, 정확한 복약은 전문가 상담이 필요합니다.")

# User Interface 실행

In [21]:
# --- UI 연결 및 실행 ---
submit_button.on_click(on_submit_clicked)
display(widgets.VBox([
    gender, age, pregnancy, mode, symptom_input, submit_button, output_box
]))


# 끝

In [ ]:
# #김지은 open ai api key사용해서 naver쇼핑 연결


# import openai

# import requests
# from urllib.parse import quote
# import json
# import re

# # ✅ OpenAI 클라이언트 초기화
# from openai import OpenAI
# client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))  # ← 본인 API 키로 교체

# # 🔐 네이버 쇼핑 API 키
# naver_client_id = "SWe77FM8c_EjAm5Y93h5"
# naver_client_secret = "mAz_OvLf7T"

# # # 🔍 네이버 쇼핑 검색 함수
# def naver_shop_search(query, display=5):
#     encoded = quote(query)
#     url = f"https://openapi.naver.com/v1/search/shop.json?query={encoded}&display={display}"
#     headers = {
#         "X-Naver-Client-Id": naver_client_id,
#         "X-Naver-Client-Secret": naver_client_secret
#     }
#     res = requests.get(url, headers=headers)
#     return res.json().get("items", [])

# # # 🧠 GPT Function Calling 정의
# function_def = {
#     "name": "recommend_nutrient",
#     "description": "증상에 따라 추천 성분 또는 제거해야 할 영양소를 제안합니다.",
#     "parameters": {
#         "type": "object",
#         "properties": {
#             "nutrient": {"type": "string", "description": "추천하거나 제거가 필요한 영양소 성분"},
#             "action": {"type": "string", "description": "'추천' 또는 '제거'"}
#         },
#         "required": ["nutrient", "action"]
#     }
# }

# # GPT 호출 함수
# def get_nutrient_action(symptom, gender, age, pregnancy, mode):
#     prompt = f"""
#     사용자 정보:
#     - 성별: {gender}
#     - 나이: {age}
#     - 임신 여부: {pregnancy}
#     - 선택 기능: {mode}

#     증상: {symptom}

#     위 조건을 고려하여 사용자에게 가장 적절한 영양소를 1가지 추천하거나, 과다 복용 시 제거가 필요한 영양소 1가지를 알려줘.
#     결과는 Function Calling 형식으로 반환해줘.
#     """
#     res = client.chat.completions.create(
#         # model="gpt-4-0613",
#         model="gpt-4o-mini",
#         messages=[
#             {"role": "system", "content": "너는 건강 기반의 영양제 조언 전문가야."},
#             {"role": "user", "content": prompt}
#         ],
#         functions=[recommend_func,search_products,google_search_tool],
#         function_call={"name": "recommend_nutrient"},
#         temperature=0.5
#     )
#     args = res.choices[0].message.function_call.arguments
#     parsed = json.loads(args)
#     return parsed.get("nutrient", "비타민"), parsed.get("action", "추천")





In [ ]:
# import ipywidgets as widgets
# from IPython.display import display, clear_output

# # --- 위젯 UI 구성 ---
# gender = widgets.ToggleButtons(options=["남성", "여성"], description='성별:')
# age = widgets.IntText(value=30, description='만 나이:')
# pregnancy = widgets.ToggleButtons(options=["해당 없음", "임신 중", "임신 가능성"], description='임신 여부:')
# mode = widgets.ToggleButtons(options=["식습관 분석을 통한 영양제 추천", "불편 현상에 따른 영양제 추천", "영양제 과다 복용 확인"], description='메뉴:')

# # ✅ 입력창 너비 확장
# symptom_input = widgets.Text(
#     placeholder='증상 또는 복용 제품 입력',
#     description='입력:',
#     layout=widgets.Layout(width='95%')
# )

# submit_button = widgets.Button(description="제출", button_style="success")
# output_box = widgets.Output()

# # --- 실행 로직 ---
# def on_submit_clicked(b):
#     with output_box:
#         clear_output()
#         info = {
#             "성별": gender.value,
#             "나이": age.value,
#             "임신 여부": pregnancy.value,
#             "선택 기능": mode.value
#         }
#         symptom = symptom_input.value.strip()

#         print(f"사용자 정보: {info}")
#         print(f"선택한 기능: {mode.value}")
#         print()

#         if not symptom:
#             print("⚠️ 증상이나 복용 제품을 입력해주세요.")
#             return

#         nutrient, action = get_nutrient_action(symptom, gender.value, age.value, pregnancy.value, mode.value)

#         if action == "추천":
#             print(f"🧾 '{symptom}'에 대한 AI 추천 결과:")
#             print(f"- {nutrient} 성분이 포함된 제품이 적합할 수 있습니다.\n")
#             products = naver_shop_search(nutrient)
#             for p in products:
#                 title = re.sub(r"<.*?>", "", p['title'])
#                 print(f"• {title}")
#                 print(f"  - 브랜드: {p.get('brand', '정보 없음')}")
#                 print(f"  - 가격: {int(p['lprice']):,}원")
#                 print(f"  - 링크: {p['link']}\n")
#         else:
#             print(f"⚠️ '{nutrient}' 성분이 중복 또는 과다 복용 위험이 있습니다.")
#             print("🔍 해당 성분이 포함된 제품을 확인하고 복용 여부를 재검토하세요.\n")
#             products = naver_shop_search(nutrient)
#             for p in products:
#                 title = re.sub(r"<.*?>", "", p['title'])
#                 print(f"• {title}")
#                 print(f"  - 브랜드: {p.get('brand', '정보 없음')}")
#                 print(f"  - 가격: {int(p['lprice']):,}원")
#                 print(f"  - 링크: {p['link']}\n")

#         print("※ 이 답변은 빅데이터 기반으로 제공되며, 정확한 복약은 전문가 상담이 필요합니다.")

In [ ]:
# # --- UI 연결 및 실행 ---
# submit_button.on_click(on_submit_clicked)
# display(widgets.VBox([

#     gender, age, pregnancy, mode, symptom_input, submit_button, output_box
# ]))
